In [1]:
# Imports for the 
import re
import csv
import sqlite3
from pathlib import Path
import pandas as pd

In [2]:
# Create database file and establish a connection
database_path = "./MPro_database.db"
conn = sqlite3.connect(database_path)
drop_order = ('compounds', 'assays')

In [3]:
# Create a database of compounds and their SMILES string
conn.execute('''
CREATE TABLE compounds
(
    CID PRIMARY KEY,
    SMILES
)
        ''')

In [4]:
df = pd.read_csv('activity_data.csv')

for i in range(len(df)):
    row = df.iloc[i]
    values  =  (row['CID'], 
         row['SMILES']
        )
    conn.execute('INSERT INTO compounds (CID, SMILES) VALUES (?,?)',
        values)

In [5]:
# Creates and populates a table of assays, including a foreign key linking the assays to the compounds table
conn.execute('''
CREATE TABLE assays
(
    CID PRIMARY KEY,
    canonical_CID,
    r_inhibition_at_20_uM,
    r_inhibition_at_50_uM,
    r_avg_IC50,
    f_inhibition_at_20_uM,
    f_inhibition_at_50_uM,
    f_avg_IC50,
    f_avg_pIC50,
    relative_solubility_at_20_uM,
    relative_solubility_at_100_uM,
    trypsin_IC50,
    NMR_std_ratio,
    acrylamide,
    chloroacetamide,
    series,
    frag_id,
    compoundID,
    FOREIGN KEY (CompoundID) REFERENCES compounds (CID)
)
        ''')


In [6]:
df_short = df.iloc[:,1:]

for i in range(len(df_short)):
    row = df.iloc[i]
    values = []
    
    for element in row[:-1]:
        values.append(element)
    print(values)
    break
    conn.execute('INSERT INTO assays (CID, canonical_CID, r_inhibition_at_20_uM, r_inhibition_at_50_uM, r_avg_IC50, f_inhibition_at_20_uM, f_inhibition_at_50_uM, f_avg_IC50, f_avg_pIC50, relative_solubility_at_20_uM, relative_solubility_at_100_uM, trypsin_IC50, NMR_std_ratio, acrylamide, chloroacetamide, series, frag_id) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',
             (values)
)

['CCn1nccc1C(=O)N1Cc2ccc(Cl)cc2C2(CCN(c3cncc4ccccc34)C2=O)C1', 'MAT-POS-be048f2c-5', 'MAT-POS-be048f2c-5', nan, nan, nan, nan, nan, 0.297012069374338, nan, nan, nan, nan, nan, False, False, '3-aminopyridine-like']


In [7]:
lis = conn.execute('''
SELECT CID
FROM assays
''')

for l in lis:
    print(l)

In [8]:
def retrieve_SMILES(compoundID: str):
    list_smiles = []
    SMILES = conn.execute('''
        SELECT SMILES
        FROM compounds
        WHERE 
            CID = ?
    ''', (compoundID,))
    for sm in SMILES:
        list_smiles.append(sm)
    return (list_smiles)
        
def retrieve_assays(compoundID: str):
    list_assays = []
    ASSAYS = conn.execute('''
        SELECT series
        FROM assays
        WHERE 
            compoundID = ?
    ''', (compoundID,))
    
    for assay in ASSAYS:
        print(assay)
        list_assays.append(assay)
    return (list_assays)
    
retrieve_assays('MAT-POS-be048f2c-5')


[]

In [9]:
# Code to display the tables currently in the database

cur = conn.execute('''
            SELECT 
                name
            FROM 
                sqlite_master 
            WHERE 
                type ='table' AND 
                name NOT LIKE 'sqlite_%'
        ''')

tables = []

for row in cur.fetchall():
    tables.append(row[0])

for table in tables:
    cur = conn.execute('SELECT  sql  FROM  sqlite_master  WHERE name=\'' + table + '\'')

    row = cur.fetchone()

    if row is not None:
        print(table+'\n')

        print(row)
    else:
        print('Table ' + table + ' not found')


compounds

('CREATE TABLE compounds\n(\n    CID PRIMARY KEY,\n    SMILES\n)',)
assays

('CREATE TABLE assays\n(\n    CID PRIMARY KEY,\n    canonical_CID,\n    r_inhibition_at_20_uM,\n    r_inhibition_at_50_uM,\n    r_avg_IC50,\n    f_inhibition_at_20_uM,\n    f_inhibition_at_50_uM,\n    f_avg_IC50,\n    f_avg_pIC50,\n    relative_solubility_at_20_uM,\n    relative_solubility_at_100_uM,\n    trypsin_IC50,\n    NMR_std_ratio,\n    acrylamide,\n    chloroacetamide,\n    series,\n    frag_id,\n    compoundID,\n    FOREIGN KEY (CompoundID) REFERENCES compounds (CID)\n)',)


In [10]:
#for table_name in drop_order:
 #   conn.execute('DROP TABLE IF EXISTS ' + table_name)